# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Setup

### Imports

In [1]:
import os
import json
import warnings
from os.path import join
from tqdm.notebook import tqdm
from itertools import repeat, starmap, product

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from numpy.linalg import norm
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from sklearn.model_selection import StratifiedGroupKFold
from kagglehub import whoami, competition_download, dataset_upload

### Supress performance warngings

In [2]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### Config

In [3]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
GRAVITY_WORLD = np.array([0, 0, 9.81], "float32")
RAW_ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
LINEAR_ACC_COLS = ["gravity_free_" + col for col in RAW_ACCELRATION_COLS]
COMPETITION_HANDLE = "cmi-detect-behavior-with-sensor-data"
CATEGORY_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
DATASET_DF_DTYPES = {
    "acc_x": "float32", "acc_y": "float32", "acc_z": "float32",
    "thm_1":"float32", "thm_2":"float32", "thm_3":"float32", "thm_4":"float32", "thm_5":"float32",
    "sequence_counter": "int32",
    **{col: "category" for col in CATEGORY_COLUMNS},
    **{f"tof_{i_1}_v{i_2}": "float32" for i_1, i_2 in product(range(1, 5), range(64))},
}
PREPROCESSED_DATASET_HANDLE = "mauroabidalcarrer/prepocessed-cmi-2025"
# The quantile of the sequences len used to pad/truncate during preprocessing
SEQUENCE_NORMED_LEN_QUANTILE = 0.95
# SAMPLING_FREQUENCY = 10 #Hz
N_FOLDS = 5
VALIDATION_FRACTION = 0.2
TARGET_NAMES = sorted([
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
])
EPSILON=1e-8
DELTA_ROTATION_ANGULAR_VELOCITY_COLS = ["angular_vel_x", "angular_vel_y", "angular_vel_z"]
DELTA_ROTATION_AXES_COLS = ["rotation_axis_x", "rotation_axis_y", "rotation_axis_z"]
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]
TOF_AGG_FUNCS = [
    "mean",
    "std",
    "min",
    "max",
    "median",
]

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [4]:
def get_feature_cols(df:DF) -> list[str]:
    return sorted(list(set(df.columns) - set(META_DATA_COLUMNS) - set(TARGET_NAMES)))

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [5]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

## Data preprocessing

### Impute missing data
Perform forward, backward and then filling of all NaN sequences.

In [6]:
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.
def get_fillna_val_per_feature_col(df:DF) -> dict:
    return {col: 1.0 if col == 'rot_w' else 0 for col in get_feature_cols(df)}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    # fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(get_fillna_val_per_feature_col(df))
    )
    return df

df = imputed_features(df)

### Norm quaternions
This allows us to parse quaternions with `scipy.spatial.transform.Rotation`.

In [7]:
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Linear acceleration
Remove gravity from the acceleration features.

In [8]:
def add_linear_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations:Rotation = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True).astype("float32")
    df[LINEAR_ACC_COLS] = df[RAW_ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_linear_acc_cols(df)

### Accelerations magnitudes
Add magnitue (norm) of both  raw acceleration and linear accelration.

In [9]:
def add_acc_magnitude(df:DF, acc_cols:list[str], acc_mag_col_name:str) -> DF:
    return df.assign(**{acc_mag_col_name: np.linalg.norm(df.loc[:, acc_cols], axis=1)})

df = add_acc_magnitude(df, RAW_ACCELRATION_COLS, "raw_acc_mag")
df = add_acc_magnitude(df, LINEAR_ACC_COLS, "linear_acc_mag")

### Quaternion angle magnitude
Refered to as "rotation angle" in top notebooks.

In [10]:
def add_quat_angle_mag(df:DF) -> DF:
    return df.assign(quat_rot_mag=np.arccos(df["rot_w"]) * 2)

df = add_quat_angle_mag(df)

### Angular velocity
Compute the axis of the rotation difference and its angular magnutide.  
Top notebooks scale the vector by the angular magnitude.  
I will also include the magnitude and unit axis vector in case they turn out to be more informative.   

In [11]:
def add_angular_velocity_features(df:DF) -> DF:
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    delta_rotations = rotations[1:] * rotations[:-1].inv()
    delta_rot_velocity = delta_rotations.as_rotvec()
    # Add extra line to avoid shape mismatch
    delta_rot_velocity = np.vstack((np.zeros((1, 3)), delta_rot_velocity))
    delta_rot_magnitude = norm(delta_rot_velocity, axis=1, keepdims=True)
    delta_rot_axes = delta_rot_velocity / (delta_rot_magnitude + EPSILON)
    df[DELTA_ROTATION_ANGULAR_VELOCITY_COLS] = delta_rot_velocity
    df[DELTA_ROTATION_AXES_COLS] = delta_rot_axes
    df["delta_rot_mag"] = delta_rot_magnitude.squeeze()

    return df

df = add_angular_velocity_features(df)

### Euler angles from quaternions

In [12]:
def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df

df = rot_euler_angles(df)

### One hot encode target values.

In [13]:
one_hot_target = pd.get_dummies(df["gesture"])
df[TARGET_NAMES] = one_hot_target[TARGET_NAMES]
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Glasses on/off,Neck - pinch skin,Neck - scratch,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Text on phone,Wave hello,Write name in air,Write name on leg
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,False,False,False,False,True
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,False,False,False,False,True
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,False,False,False,False,True
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,False,False,False,False,True


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [14]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in tqdm(range(1, 6)):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = pd.concat(
            (
                df.drop(columns=tof_cols),
                # For some reasons, it's faster to call all the aggregation functions seperatly than agg(list of functions)
                df[tof_cols].mean(axis="columns").to_frame(tof_name + "_mean"),
                df[tof_cols].std(axis="columns").to_frame(tof_name + "_std"),
                df[tof_cols].median(axis="columns").to_frame(tof_name + "_median"),
                df[tof_cols].min(axis="columns").to_frame(tof_name + "_min"),
                df[tof_cols].max(axis="columns").to_frame(tof_name + "_max"),
            ),
            axis="columns",
        )
    return df

df = agg_tof_cols_per_sensor(df)
df

  0%|          | 0/5 [00:00<?, ?it/s]

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_4_mean,tof_4_std,tof_4_median,tof_4_min,tof_4_max,tof_5_mean,tof_5_std,tof_5_median,tof_5_min,tof_5_max
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,123.359375,41.862347,134.0,51.0,206.0,135.343750,32.397930,128.5,88.0,226.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,124.406250,40.487785,135.5,60.0,206.0,137.000000,32.489803,129.5,88.0,226.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,125.687500,36.980423,131.5,71.0,206.0,140.234375,34.100105,136.0,88.0,226.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,149.078125,29.778711,147.0,100.0,210.0,142.609375,35.809734,137.5,88.0,226.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,163.765625,29.487551,157.5,116.0,229.0,151.265625,38.821220,142.5,88.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,74.062500,28.283079,66.5,39.0,162.0,52.843750,9.838730,52.0,27.0,73.0
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,70.406250,29.312611,61.5,38.0,162.0,54.531250,10.579958,54.5,29.0,73.0
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,70.109375,29.322031,63.0,41.0,162.0,57.468750,11.371042,57.5,26.0,80.0
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,75.609375,27.347044,68.0,48.0,162.0,54.937500,10.818671,54.5,25.0,77.0


In [15]:
df.dtypes.to_dict()

{'row_id': CategoricalDtype(categories=['SEQ_000007_000000', 'SEQ_000007_000001',
                   'SEQ_000007_000002', 'SEQ_000007_000003',
                   'SEQ_000007_000004', 'SEQ_000007_000005',
                   'SEQ_000007_000006', 'SEQ_000007_000007',
                   'SEQ_000007_000008', 'SEQ_000007_000009',
                   ...
                   'SEQ_065531_000043', 'SEQ_065531_000044',
                   'SEQ_065531_000045', 'SEQ_065531_000046',
                   'SEQ_065531_000047', 'SEQ_065531_000048',
                   'SEQ_065531_000049', 'SEQ_065531_000050',
                   'SEQ_065531_000051', 'SEQ_065531_000052'],
 , ordered=False, categories_dtype=object),
 'sequence_type': CategoricalDtype(categories=['Non-Target', 'Target'], ordered=False, categories_dtype=object),
 'sequence_id': CategoricalDtype(categories=['SEQ_000007', 'SEQ_000008', 'SEQ_000013', 'SEQ_000016',
                   'SEQ_000018', 'SEQ_000022', 'SEQ_000033', 'SEQ_000034',
            

In [16]:
get_fillna_val_per_feature_col(df)

{'acc_x': 0,
 'acc_y': 0,
 'acc_z': 0,
 'angular_vel_x': 0,
 'angular_vel_y': 0,
 'angular_vel_z': 0,
 'delta_rot_mag': 0,
 'euler_x': 0,
 'euler_y': 0,
 'euler_z': 0,
 'gravity_free_acc_x': 0,
 'gravity_free_acc_y': 0,
 'gravity_free_acc_z': 0,
 'linear_acc_mag': 0,
 'quat_rot_mag': 0,
 'raw_acc_mag': 0,
 'rot_w': 1.0,
 'rot_x': 0,
 'rot_y': 0,
 'rot_z': 0,
 'rotation_axis_x': 0,
 'rotation_axis_y': 0,
 'rotation_axis_z': 0,
 'thm_1': 0,
 'thm_2': 0,
 'thm_3': 0,
 'thm_4': 0,
 'thm_5': 0,
 'tof_1_max': 0,
 'tof_1_mean': 0,
 'tof_1_median': 0,
 'tof_1_min': 0,
 'tof_1_std': 0,
 'tof_2_max': 0,
 'tof_2_mean': 0,
 'tof_2_median': 0,
 'tof_2_min': 0,
 'tof_2_std': 0,
 'tof_3_max': 0,
 'tof_3_mean': 0,
 'tof_3_median': 0,
 'tof_3_min': 0,
 'tof_3_std': 0,
 'tof_4_max': 0,
 'tof_4_mean': 0,
 'tof_4_median': 0,
 'tof_4_min': 0,
 'tof_4_std': 0,
 'tof_5_max': 0,
 'tof_5_mean': 0,
 'tof_5_median': 0,
 'tof_5_min': 0,
 'tof_5_std': 0}

### Add derivatives w.r.t time features

In [ ]:
def add_diff_features(df:DF) -> DF:
    return pd.concat(
        (
            df,
            df
            .groupby("sequence_id", as_index=False, observed=True)
            [get_feature_cols(df)]
            .diff()
            .fillna(get_fillna_val_per_feature_col(df))
            .add_prefix("diff_")
        ),
        axis="columns",
    )

df = add_diff_features(df)

In [18]:
len(get_feature_cols(df))

106

### Split into folds

In [19]:
df["gesture"].unique().tolist()

['Cheek - pinch skin',
 'Forehead - pull hairline',
 'Write name on leg',
 'Feel around in tray and pull out an object',
 'Neck - scratch',
 'Neck - pinch skin',
 'Eyelash - pull hair',
 'Eyebrow - pull hair',
 'Forehead - scratch',
 'Above ear - pull hair',
 'Wave hello',
 'Write name in air',
 'Text on phone',
 'Pull air toward your face',
 'Pinch knee/leg skin',
 'Scratch knee/leg skin',
 'Drink from bottle/cup',
 'Glasses on/off']

In [20]:
def split_dataset_stratified_groupkfold(df: pd.DataFrame, target_col: str, group_col: str) -> list[tuple[pd.DataFrame, pd.DataFrame]]:
    sgkf = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True)

    folds = []
    X = df.drop(columns=[target_col])
    y = df[target_col]
    groups = df[group_col]

    for train_idx, valid_idx in sgkf.split(X, y, groups):
        train_df = df.iloc[train_idx].copy()
        valid_df = df.iloc[valid_idx].copy()
        folds.append((train_df, valid_df))

    return folds

# Example usage:
folds = split_dataset_stratified_groupkfold(df, target_col="gesture", group_col="subject")

### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [23]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[get_feature_cols(df)].mean().astype("float32")
    stds = train[get_feature_cols(df)].std().astype("float32")
    test = train[get_feature_cols(df)] - means
    train.loc[:, get_feature_cols(df)] = (train[get_feature_cols(df)] - means) / stds
    val.loc[:, get_feature_cols(df)] = (val[get_feature_cols(df)] - means) / stds
    return train, val

folds = list(starmap(std_norm_dataset, folds))

Normalize full dataset.

In [24]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[get_feature_cols(df)].mean().astype("float32").to_dict(),
    "std": df[get_feature_cols(df)].std().astype("float32").to_dict(),
}
df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

/tmp/ipykernel_36489/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.87240755 0.91835342 0.70619159 ... 0.24943555 0.40213801 0.45484063]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_36489/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.8841304   0.8841304   0.72331979 ... -0.31414283 -0.42967666
 -0.54442986]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_36489/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated

Verify the mean and std of the full dataset.

In [25]:
df[get_feature_cols(df)].agg(["mean", "std"])

,acc_x,acc_y,acc_z,angular_vel_x,angular_vel_y,angular_vel_z,delta_rot_mag,diff_acc_x,diff_acc_y,diff_acc_z,...,tof_4_max,tof_4_mean,tof_4_median,tof_4_min,tof_4_std,tof_5_max,tof_5_mean,tof_5_median,tof_5_min,tof_5_std
mean,-1.749467e-08,1.186949e-08,-2.971537e-09,8.305834e-10,-5.170893e-10,5.603417e-10,-2.484010e-09,-3.192562e-10,-1.666452e-10,-3.622584e-10,...,-6.377776e-08,1.404808e-08,-9.119742e-08,-1.658729e-08,2.203446e-07,5.033017e-08,-5.993022e-08,-2.861646e-08,1.314192e-08,-4.774061e-08
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [26]:
pd.concat({
    "train": folds[0][0][get_feature_cols(df)].agg(["mean", "std"]),
    "validation": folds[0][1][get_feature_cols(df)].agg(["mean", "std"]),
})

acc_x         acc_y         acc_z  angular_vel_x  \
train      mean -8.198549e-09 -1.996743e-08  6.347264e-09   7.742488e-10   
           std   9.999999e-01  9.999999e-01  1.000000e+00   1.000000e+00   
validation mean -9.726086e-02  7.166609e-02  7.997745e-03  -6.601994e-03   
           std   9.914509e-01  9.994850e-01  1.003435e+00   1.007533e+00   

                 angular_vel_y  angular_vel_z  delta_rot_mag    diff_acc_x  \
train      mean   2.944432e-10  -5.791696e-10  -2.958919e-09 -8.677900e-10   
           std    1.000000e+00   1.000000e+00   1.000000e+00  1.000000e+00   
validation mean  -1.053239e-02   1.028517e-02   7.974006e-03  6.403131e-03   
           std    1.010691e+00   1.021054e+00   1.011692e+00  9.110948e-01   

                   diff_acc_y    diff_acc_z  ...     tof_4_max    tof_4_mean  \
train      mean  1.024819e-09  2.776928e-09  ... -9.785365e-08 -3.808358e-08   
           std   1.000000e+00  1.000000e+00  ...  1.000000e+00  1.000000e+00   
validation mean  1.074796e-03 -2.484413e-03  ...  2.538789e-02 -2.322527e-02   
           std   9.842595e-01  9.685237e-01  ...  9.607834e-01  9.615051e-01   

                 tof_4_median     tof_4_min     tof_4_std     tof_5_max  \
train      mean  2.750481e-08 -1.481028e-08 -2.479400e-08  7.669610e-08   
           std   1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -2.716631e-02 -9.683621e-02  6.497943e-02  1.718117e-01   
           std   9.832078e-01  9.586236e-01  9.906979e-01  7.230130e-01   

                   tof_5_mean  tof_5_median     tof_5_min     tof_5_std  
train      mean  1.057877e-07 -7.471258e-08  6.109241e-08  2.909163e-08  
           std   9.999999e-01  1.000000e+00  1.000000e+00  1.000000e+00  
validation mean  7.978355e-02  5.957159e-02 -2.309146e-02  1.754321e-01  
           std   8.546929e-01  9.261847e-01  9.751214e-01  9.565742e-01  

[4 rows x 106 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [27]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [28]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [29]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 127
validation normed sequence len: 122

train normed sequence len: 124
validation normed sequence len: 136

train normed sequence len: 129
validation normed sequence len: 122

train normed sequence len: 129
validation normed sequence len: 121

train normed sequence len: 126
validation normed sequence len: 127



#### Sequence length norm implementation

In [30]:
def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, get_feature_cols(df)]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(get_feature_cols(df))),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), len(TARGET_NAMES)),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[TARGET_NAMES].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    # normed_sequence_len = get_normed_seq_len(train)
    full_dataset_normed_seq_len = get_normed_seq_len(df)
    return (
        *df_to_ndarrays(train, full_dataset_normed_seq_len),
        *df_to_ndarrays(validation, full_dataset_normed_seq_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6466 [00:00<?, ?it/s]

  0%|          | 0/1685 [00:00<?, ?it/s]

  0%|          | 0/6524 [00:00<?, ?it/s]

  0%|          | 0/1627 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6576 [00:00<?, ?it/s]

  0%|          | 0/1575 [00:00<?, ?it/s]

In [31]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [32]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
# full_dataset_meta_data["target_names"] = TARGET_NAMES
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["feature_cols"] = get_feature_cols(df)

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [33]:
df[get_feature_cols(df)]

,acc_x,acc_y,acc_z,angular_vel_x,angular_vel_y,angular_vel_z,delta_rot_mag,diff_acc_x,diff_acc_y,diff_acc_z,...,tof_4_max,tof_4_mean,tof_4_median,tof_4_min,tof_4_std,tof_5_max,tof_5_mean,tof_5_median,tof_5_min,tof_5_std
0,0.872408,0.884130,0.625816,-0.021278,-0.010010,-0.013910,-0.458822,0.015724,-0.027387,0.036749,...,-0.129341,-0.016778,0.164689,0.317342,-0.261427,0.230914,0.404678,0.263911,1.791887,-0.736622
1,0.918353,0.884130,0.588012,-0.075474,-0.238310,-0.191092,-0.269318,0.139234,-0.027387,-0.075917,...,-0.129341,0.002596,0.187552,0.536344,-0.314810,0.230914,0.433771,0.278181,1.791887,-0.733320
2,0.706192,0.723320,0.964766,-0.554449,-1.075163,-0.461610,0.376415,-0.554598,-0.549341,1.159588,...,-0.129341,0.026308,0.126584,0.804014,-0.451020,0.230914,0.490583,0.370941,1.791887,-0.675448
3,0.858218,0.347835,1.134561,-0.961076,-0.665305,-0.944204,0.467370,0.424395,-1.246124,0.542790,...,-0.063366,0.459197,0.362833,1.509689,-0.730704,0.230914,0.532300,0.392347,1.791887,-0.614007
4,0.679165,-0.302433,1.655481,-0.618189,-1.052999,-1.509886,0.738559,-0.465599,-2.138006,1.589246,...,0.250015,0.731018,0.522873,1.899026,-0.742011,0.230914,0.684347,0.463701,1.791887,-0.505779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,0.322408,-0.444509,-1.309212,-0.231978,0.171270,-0.070187,-0.275002,-0.020602,0.172780,0.409119,...,-0.855065,-0.929112,-0.864138,0.025339,-0.788788,-1.915469,-1.044439,-0.827803,0.062303,-1.547361
574941,0.369030,-0.490566,-1.434796,-0.119604,-0.136954,-0.060574,-0.349591,0.141050,-0.176879,-0.337531,...,-0.855065,-0.996778,-0.940347,0.001005,-0.748805,-1.915469,-1.014798,-0.792126,0.119011,-1.520723
574942,0.249436,-0.314143,-1.138775,-0.010458,-0.038358,0.003139,-0.436163,-0.305763,0.545242,0.918980,...,-0.855065,-1.002272,-0.917485,0.074006,-0.748439,-1.817268,-0.963200,-0.749314,0.033949,-1.492293
574943,0.402138,-0.429677,-1.414933,-0.118555,-0.138399,-0.056625,-0.349784,0.426211,-0.402383,-0.786284,...,-0.855065,-0.900484,-0.841275,0.244341,-0.825139,-1.859354,-1.007662,-0.792126,0.005595,-1.512144


## Dataset upload
Optionally upload the dataset to kaggle.

In [34]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
